# atmodeller

## Tutorial 5: Si in the atmosphere via magma-atmosphere chemistry


In [ ]:
import logging
from typing import Type

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from atmodeller import __version__, debug_logger
from atmodeller.constraints import (
    FugacityConstraint,
    IronWustiteBufferConstraintHirschmann,
    MassConstraint,
    TotalPressureConstraint,
    SystemConstraints,
)
from atmodeller.eos.holland import (
    get_holland_eos_models,
)
from atmodeller.interfaces import (
    GasSpecies,
    LiquidSpecies,
    IdealGas,
    NoSolubility,
    RealGasABC,
)
from atmodeller.interior_atmosphere import InteriorAtmosphereSystem, Planet, Species
from atmodeller.solubilities import BasaltDixonCO2, BasaltH2, PeridotiteH2O
from atmodeller.utilities import earth_oceans_to_kg

#logger: logging.Logger = debug_logger()


eos_models: dict[str, RealGasABC] = get_holland_eos_models()

rtol: float = 1.0e-8
atol: float = 1.0e-8


### Functions to setup and solve the system and store the output

In [ ]:
def setup_species(
        flag_solubility = False, # If True: solubility of H2 and H2O considered
        flag_nonideality = False, # If True: non-ideality of H2 and H2O considered
        ):
    if flag_solubility == False and flag_nonideality == False:
        species: Species = Species(
            [
                GasSpecies(
                    chemical_formula="H2",
                    solubility=NoSolubility(),
                #    eos=eos_models["H2"],
                ),
                GasSpecies(
                    chemical_formula="H2O",
                    solubility=NoSolubility(),
                #    eos=eos_models["H2O"],
                ),
                GasSpecies(
                    chemical_formula="O2",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="OSi",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="H4Si",
                    solubility=NoSolubility(),
                ),
                LiquidSpecies(
                    chemical_formula="O2Si",
                    name_in_thermodynamic_data="O2Si(l)",
                ),
            ]
        )
    
    elif flag_solubility == True and flag_nonideality == False:
        species: Species = Species(
            [
                GasSpecies(
                    chemical_formula="H2",
                    solubility=BasaltH2(),
                #    eos=eos_models["H2"],
                ),
                GasSpecies(
                    chemical_formula="H2O",
                    solubility=PeridotiteH2O(),
                #    eos=eos_models["H2O"],
                ),
                GasSpecies(
                    chemical_formula="O2",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="OSi",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="H4Si",
                    solubility=NoSolubility(),
                ),
                LiquidSpecies(
                    chemical_formula="O2Si",
                    name_in_thermodynamic_data="O2Si(l)",
                ),
            ]
        )

    elif flag_solubility == False and flag_nonideality == True:
        species: Species = Species(
            [
                GasSpecies(
                    chemical_formula="H2",
                    solubility=NoSolubility(),
                    eos=eos_models["H2"],
                ),
                GasSpecies(
                    chemical_formula="H2O",
                    solubility=NoSolubility(),
                    eos=eos_models["H2O"],
                ),
                GasSpecies(
                    chemical_formula="O2",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="OSi",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="H4Si",
                    solubility=NoSolubility(),
                ),
                LiquidSpecies(
                    chemical_formula="O2Si",
                    name_in_thermodynamic_data="O2Si(l)",
                ),
            ]
        )

    elif flag_solubility == True and flag_nonideality == True:
        species: Species = Species(
            [
                GasSpecies(
                    chemical_formula="H2",
                    solubility=BasaltH2(),
                    eos=eos_models["H2"],
                ),
                GasSpecies(
                    chemical_formula="H2O",
                    solubility=PeridotiteH2O(),
                    eos=eos_models["H2O"],
                ),
                GasSpecies(
                    chemical_formula="O2",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="OSi",
                    solubility=NoSolubility(),
                ),
                GasSpecies(
                    chemical_formula="H4Si",
                    solubility=NoSolubility(),
                ),
                LiquidSpecies(
                    chemical_formula="O2Si",
                    name_in_thermodynamic_data="O2Si(l)",
                ),
            ]
        )

    return species

def setup_planet(
        surface_temperature=3400, # kelvin
        mantle_mass=11*4.208261222595111e24, # kg
        surface_radius=1.7*6371000, # metre
        ):
    return Planet(surface_temperature=surface_temperature, mantle_mass=mantle_mass, 
                  surface_radius=surface_radius)

def setup_constraints(
        flag_mass_constraint = True, # if True: MassConstraint(), else: TotalPressureConstraint()
        massH_ocean = 200, # mass of H equivalent to water oceans
        total_pressure = 100000, # bar
        logfO2_shift = 0, # deltaIW = 0 by default
        ):
    
    if flag_mass_constraint == True:
        h_kg: float = earth_oceans_to_kg(massH_ocean)
        constraint: SystemConstraints = SystemConstraints(
            [
                IronWustiteBufferConstraintHirschmann(log10_shift=logfO2_shift),
                MassConstraint(species="H", value=h_kg),
            ]
        )
    else:
        constraint: SystemConstraints = SystemConstraints(
            [
                IronWustiteBufferConstraintHirschmann(log10_shift=logfO2_shift),
                TotalPressureConstraint(value=total_pressure),
            ]
        )
    return constraint

def setup_system(
        species, 
        planet,
        ):
    return InteriorAtmosphereSystem(species=species, planet=planet)

def solve_system(
        system,
        constraint,
        factor=0.1,
        ):
    initial_solution: np.ndarray = np.array([100, 1000, 1e-8, 100, 10, 1000])
    system.solve(constraint, factor=factor, tol=1e-10, initial_solution=initial_solution)
    solution = system.solution_dict
    solution['total_pressure'] = system.total_pressure
    solution['log_fO2_dIW'] = np.log10(solution['O2']) \
    - IronWustiteBufferConstraintHirschmann().get_buffer_log10_value( \
        temperature=system.planet.surface_temperature, pressure=system.total_pressure)

    return solution

### 0. Problematic Example

In [ ]:
logger: logging.Logger = debug_logger()

species = setup_species(flag_solubility = False, flag_nonideality = True)
planet = setup_planet(surface_temperature = 3400, mantle_mass = 11*4.208261222595111e24, 
                      surface_radius = 1.7*6371000)

constraint = setup_constraints(
    flag_mass_constraint = True, # if True: MassConstraint(), else: TotalPressureConstraint()
    massH_ocean = 200, # mass of H equivalent to water oceans # Used only if prior flag is True
    total_pressure = 100000, # bar # Used only if prior flag is False
    logfO2_shift = -8, # deltaIW = 0 by default
    )

system = setup_system(species, planet)

solution = solve_system(system, constraint, factor=0.1)
system.output

### 1a. H2/H2O ratio variation: H2O-H2-O2-SiO-SiH4-SiO2l system (ideal, no solubility) 

In [ ]:
species = setup_species(flag_solubility = False, flag_nonideality = False)
#planet = setup_planet(surface_temperature = 3400, mantle_mass = 1*4.208261222595111e24, 
#                      surface_radius = 1*6371000)
planet = setup_planet(surface_temperature = 3400, mantle_mass = 11*4.208261222595111e24, 
                      surface_radius = 1.7*6371000)

log_fO2_dIWs = np.linspace(-9, 2, num=100) 

constraints = []

for log_fO2_dIW in log_fO2_dIWs:

    constraint = setup_constraints(
        flag_mass_constraint = True, # if True: MassConstraint(), else: TotalPressureConstraint()
        massH_ocean = 200, # mass of H equivalent to water oceans
        total_pressure = 100000, # bar
        logfO2_shift = log_fO2_dIW, # deltaIW = 0 by default
        )
    constraints.append(constraint)

system = setup_system(species, planet)
solutions = []

for (log_fO2_dIW, constraint) in zip(log_fO2_dIWs, constraints):

    solution = solve_system(system, constraint, factor=0.1)
    solutions.append(solution)

filename = "atmodeller_SiOH_tutorial5.csv"
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv(filename) 
ratio = df['H2'].values / df['H2O'].values
logfO2 = df['log_fO2_dIW'].values
H2 = df['H2'].values
H2O = df['H2O'].values
SiH4 = df['H4Si'].values
SiO = df['OSi'].values
O2 = df['O2'].values
tot = df['total_pressure'].values

fig, ax1 = plt.subplots(1, figsize=(6,4), tight_layout='True')

ax1.set_ylim([1e-10,1.2])
ax1.axvspan(2.6e-2, 1e0, alpha=0.2, color='blue')
ax1.axvspan(1e0, 2e2, alpha=0.2, color='green')
ax1.axvspan(2e2, 3e3, alpha=0.2, color='red')

ax1a = ax1.twinx()
ax2 = ax1a.twiny()
ax2.invert_xaxis()
ax2.plot(logfO2, tot/1000, color='black', lw=2, label=r'$P_{\rm surface}$', zorder=0)

ax1.text(3e-2, 1e-3, 'Steam Worlds', fontsize=10)
ax1.text(3, 1e-5, 'Hydrogen Worlds', fontsize=10)
ax1.text(2e2, 1e-7, 'Silicon Worlds', fontsize=10)

ax1.plot(ratio, H2/tot, color='orange', lw=2, label=r'H$_2$')
ax1.plot(ratio, H2O/tot, color='blue', lw=2, label=r'H$_2$O')
ax1.plot(ratio, SiH4/tot, color='red', lw=2, label=r'SiH$_4$')
ax1.plot(ratio, SiO/tot, color='brown', lw=2, label=r'SiO')

ax1.set_title('Atmospheric Composition at the Surface at 3400 K')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$P_{\rm H_2}/P_{\rm H_2O}$', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)

ax2.set_xlabel(r"log $f_{\rm O_2}$ ($\Delta$IW)", fontsize=14)
ax2.set_ylabel(r"$P_{\rm total}$ [kbar]", fontsize=14)
ax2.set_ylim([0,1.1e2])


ax1a.set_ylabel(r"$P_{\rm surface}$ [kbar]", fontsize=14)
ax1.legend(ncol=2, loc='lower right')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_ideal.pdf', bbox_inches='tight')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_ideal.png', bbox_inches='tight')
plt.show()

### 1b. H2/H2O ratio variation: H2O-H2-O2-SiO-SiH4-SiO2l system (ideal, solubility) 

In [ ]:
species = setup_species(flag_solubility = True, flag_nonideality = False)
planet = setup_planet(surface_temperature = 3400, mantle_mass = 11*4.208261222595111e24, 
                      surface_radius = 1.7*6371000)

log_fO2_dIWs = np.linspace(-9, 2, num=100) 

constraints = []

for log_fO2_dIW in log_fO2_dIWs:

    constraint = setup_constraints(
        flag_mass_constraint = False, # if True: MassConstraint(), else: TotalPressureConstraint()
        massH_ocean = 200, # mass of H equivalent to water oceans
        total_pressure = 100000, # bar
        logfO2_shift = log_fO2_dIW, # deltaIW = 0 by default
        )
    constraints.append(constraint)

system = setup_system(species, planet)
solutions = []

for (log_fO2_dIW, constraint) in zip(log_fO2_dIWs, constraints):

    solution = solve_system(system, constraint, factor=0.1)
    solutions.append(solution)

filename = "atmodeller_SiOH_tutorial5_ideal_solubility.csv"
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv(filename) 
ratio = df['H2'].values / df['H2O'].values
logfO2 = df['log_fO2_dIW'].values
H2 = df['H2'].values
H2O = df['H2O'].values
SiH4 = df['H4Si'].values
SiO = df['OSi'].values
O2 = df['O2'].values
tot = df['total_pressure'].values

fig, ax1 = plt.subplots(1, figsize=(6,4), tight_layout='True')

ax1.set_ylim([1e-10,1.2])
ax1.axvspan(2.6e-2, 1e0, alpha=0.2, color='blue')
ax1.axvspan(1e0, 2e2, alpha=0.2, color='green')
ax1.axvspan(2e2, 3e3, alpha=0.2, color='red')

ax1a = ax1.twinx()
ax2 = ax1a.twiny()
ax2.invert_xaxis()
ax2.plot(logfO2, tot/1000, color='black', lw=2, label=r'$P_{\rm surface}$', zorder=0)

ax1.text(3e-2, 1e-3, 'Steam Worlds', fontsize=10)
ax1.text(3, 1e-5, 'Hydrogen Worlds', fontsize=10)
ax1.text(2e2, 1e-7, 'Silicon Worlds', fontsize=10)

ax1.plot(ratio, H2/tot, color='orange', lw=2, label=r'H$_2$')
ax1.plot(ratio, H2O/tot, color='blue', lw=2, label=r'H$_2$O')
ax1.plot(ratio, SiH4/tot, color='red', lw=2, label=r'SiH$_4$')
ax1.plot(ratio, SiO/tot, color='brown', lw=2, label=r'SiO')

ax1.set_title('Atmospheric Composition at the Surface at 3400 K')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$P_{\rm H_2}/P_{\rm H_2O}$', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)

ax2.set_xlabel(r"log $f_{\rm O_2}$ ($\Delta$IW)", fontsize=14)
ax2.set_ylabel(r"$P_{\rm total}$ [kbar]", fontsize=14)
ax2.set_ylim([0,1.1e2])

ax1a.set_ylabel(r"$P_{\rm surface}$ [kbar]", fontsize=14)
ax1.legend(ncol=2, loc='lower right')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_ideal_solubility.pdf', bbox_inches='tight')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_ideal_solubility.png', bbox_inches='tight')
plt.show()

### 1c. H2/H2O ratio variation: H2O-H2-O2-SiO-SiH4-SiO2l system (non-ideal, no solubility) 

In [ ]:
species = setup_species(flag_solubility = False, flag_nonideality = True)
planet = setup_planet(surface_temperature = 3400, mantle_mass = 11*4.208261222595111e24, 
                      surface_radius = 1.7*6371000)

log_fO2_dIWs = np.linspace(-8, 2, num=100) 

constraints = []

for log_fO2_dIW in log_fO2_dIWs:

    constraint = setup_constraints(
        flag_mass_constraint = True, # if True: MassConstraint(), else: TotalPressureConstraint()
        massH_ocean = 200, # mass of H equivalent to water oceans
        total_pressure = 10000, # bar
        logfO2_shift = log_fO2_dIW, # deltaIW = 0 by default
        )
    constraints.append(constraint)

system = setup_system(species, planet)
solutions = []

for (log_fO2_dIW, constraint) in zip(log_fO2_dIWs, constraints):

    solution = solve_system(system, constraint, factor=0.1)
    solutions.append(solution)

filename = "atmodeller_SiOH_tutorial5_nonideal.csv"
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv(filename) 
ratio = df['H2'].values / df['H2O'].values
logfO2 = df['log_fO2_dIW'].values
H2 = df['H2'].values
H2O = df['H2O'].values
SiH4 = df['H4Si'].values
SiO = df['OSi'].values
O2 = df['O2'].values
tot = df['total_pressure'].values

fig, ax1 = plt.subplots(1, figsize=(6,4), tight_layout='True')

ax1.set_ylim([1e-10,1.2])
ax1.axvspan(2.6e-2, 1e0, alpha=0.2, color='blue')
ax1.axvspan(1e0, 2e2, alpha=0.2, color='green')
ax1.axvspan(2e2, 3e3, alpha=0.2, color='red')

ax1a = ax1.twinx()
ax2 = ax1a.twiny()
ax2.invert_xaxis()
ax2.plot(logfO2, tot/1000, color='black', lw=2, label=r'$P_{\rm surface}$', zorder=0)

ax1.text(3e-2, 1e-3, 'Steam Worlds', fontsize=10)
ax1.text(3, 1e-5, 'Hydrogen Worlds', fontsize=10)
ax1.text(2e2, 1e-7, 'Silicon Worlds', fontsize=10)

ax1.plot(ratio, H2/tot, color='orange', lw=2, label=r'H$_2$')
ax1.plot(ratio, H2O/tot, color='blue', lw=2, label=r'H$_2$O')
ax1.plot(ratio, SiH4/tot, color='red', lw=2, label=r'SiH$_4$')
ax1.plot(ratio, SiO/tot, color='brown', lw=2, label=r'SiO')

ax1.set_title('Atmospheric Composition at the Surface at 3400 K')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$P_{\rm H_2}/P_{\rm H_2O}$', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)

ax2.set_xlabel(r"log $f_{\rm O_2}$ ($\Delta$IW)", fontsize=14)
ax2.set_ylabel(r"$P_{\rm total}$ [kbar]", fontsize=14)

ax1a.set_ylabel(r"$P_{\rm surface}$ [kbar]", fontsize=14)
ax1.legend(ncol=2, loc='lower right')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_nonideal.pdf', bbox_inches='tight')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_nonideal.png', bbox_inches='tight')
plt.show()

### 1d. H2/H2O ratio variation: H2O-H2-O2-SiO-SiH4-SiO2l system  (non-ideal, solubility)

In [ ]:
species = setup_species(flag_solubility = True, flag_nonideality = True)
planet = setup_planet(surface_temperature = 3400, mantle_mass = 11*4.208261222595111e24, 
                      surface_radius = 1.7*6371000)

log_fO2_dIWs = np.linspace(-9, 2, num=100) 

constraints = []

for log_fO2_dIW in log_fO2_dIWs:

    constraint = setup_constraints(
        flag_mass_constraint = True, # if True: MassConstraint(), else: TotalPressureConstraint()
        massH_ocean = 200, # mass of H equivalent to water oceans
        total_pressure = 100000, # bar
        logfO2_shift = log_fO2_dIW, # deltaIW = 0 by default
        )
    constraints.append(constraint)

system = setup_system(species, planet)
solutions = []

for (log_fO2_dIW, constraint) in zip(log_fO2_dIWs, constraints):

    solution = solve_system(system, constraint, factor=0.1)
    solutions.append(solution)

filename = "atmodeller_SiOH_tutorial5_nonideal_solubility.csv"
df = pd.DataFrame(solutions)
df.to_csv(filename, encoding='utf-8', index=False)

In [ ]:
df = pd.read_csv(filename) 
ratio = df['H2'].values / df['H2O'].values
logfO2 = df['log_fO2_dIW'].values
H2 = df['H2'].values
H2O = df['H2O'].values
SiH4 = df['H4Si'].values
SiO = df['OSi'].values
O2 = df['O2'].values
tot = df['total_pressure'].values

fig, ax1 = plt.subplots(1, figsize=(6,4), tight_layout='True')

ax1.set_ylim([1e-10,1.2])
ax1.axvspan(2.6e-2, 1e0, alpha=0.2, color='blue')
ax1.axvspan(1e0, 2e2, alpha=0.2, color='green')
ax1.axvspan(2e2, 3e3, alpha=0.2, color='red')

ax1a = ax1.twinx()
ax2 = ax1a.twiny()
ax2.invert_xaxis()
ax2.plot(logfO2, tot/1000, color='black', lw=2, label=r'$P_{\rm surface}$', zorder=0)

ax1.text(3e-2, 1e-3, 'Steam Worlds', fontsize=10)
ax1.text(3, 1e-5, 'Hydrogen Worlds', fontsize=10)
ax1.text(2e2, 1e-7, 'Silicon Worlds', fontsize=10)

ax1.plot(ratio, H2/tot, color='orange', lw=2, label=r'H$_2$')
ax1.plot(ratio, H2O/tot, color='blue', lw=2, label=r'H$_2$O')
ax1.plot(ratio, SiH4/tot, color='red', lw=2, label=r'SiH$_4$')
ax1.plot(ratio, SiO/tot, color='brown', lw=2, label=r'SiO')

ax1.set_title('Atmospheric Composition at the Surface at 3400 K')

ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.set_xlabel(r'$P_{\rm H_2}/P_{\rm H_2O}$', fontsize=14)
ax1.set_ylabel(r'Mixing Ratios', fontsize=14)

ax2.set_xlabel(r"log $f_{\rm O_2}$ ($\Delta$IW)", fontsize=14)
ax2.set_ylabel(r"$P_{\rm total}$ [kbar]", fontsize=14)

ax1a.set_ylabel(r"$P_{\rm surface}$ [kbar]", fontsize=14)
ax1.legend(ncol=2, loc='lower right')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_nonideal_solubility.pdf', bbox_inches='tight')
plt.savefig('atmodeller_SiOH_fH2fH2O_200H2Oocean_3400K_nonideal_solubility.png', bbox_inches='tight')
plt.show()